In [1]:
from datetime import datetime

# import qty as qty
from pybit import HTTP
import pandas as pd
from ta.volatility import AverageTrueRange

In [2]:
# side = 'Buy'
side = 'Sell'
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
karid = False
# karid = True
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Risk = 2
USD = 1000
interval_asl = 1
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

startTime = datetime.now()
startTime = datetime.timestamp(startTime)
if type(interval_asl)==int:
    st = startTime-190*60*interval_asl
elif interval_asl=='D':
    st = startTime-190*86400

In [3]:
st

1642933407.071989

In [4]:
from pybit import HTTP
session = HTTP("https://api-testnet.bybit.com",
               api_key="qBpP1YrhKM6wWEHmJ7",
               api_secret="KI7GkmQ5cfMWBc4V9E70ew1v83WvfD1RbjMX")
df = pd.DataFrame(session.query_kline(
    symbol="BTCUSDT",
    interval=interval_asl,
    from_time=str(int(st))
)['result'])

df['dddd'] = pd.to_datetime(df.start_at,unit='s')
df['ATR'] = round(AverageTrueRange(window=14,high=df.high,low=df.low,close=df.close).average_true_range(),2)

close_last = df.iloc[-1,-4]
ATR = df.iloc[-1,-1]
qty_cal = round(USD/close_last,3)
#--------------------------------------------------------------------------------------------------

if (side == 'Buy'):
    stop_loss = close_last - 2*ATR
    print(side)
elif (side == 'Sell'):
    stop_loss = close_last + 2*ATR
else:
    print('side ro vared kon')
#-----------------------------------------------------------

Range = ((abs((close_last-stop_loss))/close_last))*100
leverage = round(Risk/Range,2)
if leverage <1 :
    leverage = 1
print(f'close      = {close_last:-^30},\n'
      f'stop_loss  = {stop_loss:-^30},\n'
      f'leverage   = {leverage:-^30},\n'
      f'hade zarar = {(abs(stop_loss-close_last)/close_last)*100:-^30}')

close      = -----------35826.0------------,
stop_loss  = -----------35858.08-----------,
leverage   = ------------22.34-------------,
hade zarar = -----0.08954390665997249------


In [110]:
session.set_leverage(
    symbol="BTCUSDT",
    buy_leverage=leverage,
    sell_leverage=leverage
)

{'ret_code': 0,
 'ret_msg': 'OK',
 'ext_code': '',
 'ext_info': '',
 'result': None,
 'time_now': '1642932023.521225',
 'rate_limit_status': 74,
 'rate_limit_reset_ms': 1642932023517,
 'rate_limit': 75}

In [111]:
#----------( close )--------------
with open('qty.text','r') as aa:
    qty = float(aa.read())
with open('SellAndBuy.text','r') as aa:
    SellAndBuy = aa.read()
if SellAndBuy=='Buy':
    SellAndBuy='Sell'
elif SellAndBuy=='Sell':
    SellAndBuy='Buy'

In [112]:
if qty>0:
    session.place_active_order(
        symbol="BTCUSDT",
        side=SellAndBuy,
        order_type="Market",
        qty=qty,
        # price=30000,
        time_in_force="GoodTillCancel",
        reduce_only=True,
        close_on_trigger=False)

    with open('qty.text','w') as aa:
        aa.write(str(0))
    with open('SellAndBuy.text','w') as aa:
        aa.write(str(0))

In [113]:
# #----------( open )--------------
if karid == True :
    order = session.place_active_order(
        symbol="BTCUSDT",
        side=side,
        order_type="Market",
        qty=qty_cal,
        # price=30000,
        time_in_force="GoodTillCancel",
        reduce_only=False,
        close_on_trigger=False,
        stop_loss =stop_loss
    )
    print('ersal shod')
    qty = order["result"]["qty"]
    SellAndBuy = order["result"]["side"]
    print('------------------------')
    print('dariaft shod')

    with open('qty.text','w') as tt:
        tt.write(str(qty))
    with open('SellAndBuy.text','w') as tt:
        tt.write(str(SellAndBuy))